# Permutation of group differences. 

In [2]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import scona as scn
from scona.visualisations import plot_network_measures, plot_degree_dist, plot_rich_club
import matplotlib.pyplot as plt
import nilearn.plotting as plot
import functions.plotting_functions as Pfun
import functions.statistical_functions as Sfun
import seaborn as sns
sns.set_style('dark')

This code creates the enviornmental variable for where the data is stored. Create a .env file in the directory with the file path to data.

In [3]:
from decouple import config

data = config('data')

In [4]:
lh_volume = pd.read_csv(f'{data}/lh_volume.dat',sep='\t').drop([
                     'BrainSegVolNotVent', 'eTIV'],axis=1).rename(columns={'lh.aparc.volume':'G-Number'})

rh_volume =  pd.read_csv(f'{data}/rh_volume.dat',sep='\t').drop([
                       'BrainSegVolNotVent', 'eTIV','rh.aparc.volume'],axis=1)

group = pd.read_csv(f'{data}/cortical_measures.csv').iloc[0:,2]

volume = pd.concat([lh_volume, rh_volume, group],axis=1)

names = list(volume.columns.drop(['G-Number','age_adjusted_group']))

centroids = pd.read_csv(f'{data}/atlas.csv')

centroids = centroids[['x.mni', 'y.mni', 'z.mni']].to_numpy()

group = volume.groupby('age_adjusted_group')
aan = group.get_group('AAN').reset_index(drop=True)
hc = group.get_group('HC').reset_index(drop=True)
wr = group.get_group('WR').reset_index(drop=True)
aan.head()

,G-Number,lh_bankssts_volume,lh_caudalanteriorcingulate_volume,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,lh_entorhinal_volume,lh_fusiform_volume,lh_inferiorparietal_volume,lh_inferiortemporal_volume,lh_isthmuscingulate_volume,...,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume,age_adjusted_group
0,sub-G2001,2885.0,1875.0,5887.0,2985.0,2195.0,8822.0,12823.0,10927.0,2617.0,...,15777.0,21509.0,13309.0,11317.0,9869.0,1616.0,3326.0,999.0,6634.0,AAN
1,sub-G2002,3685.0,1842.0,7653.0,4092.0,2116.0,11551.0,13769.0,13400.0,2909.0,...,20591.0,23157.0,14659.0,14117.0,8507.0,1640.0,2310.0,1084.0,7527.0,AAN
2,sub-G2003,3865.0,2396.0,6341.0,2826.0,2120.0,11673.0,13934.0,12608.0,2553.0,...,17216.0,23072.0,13261.0,12634.0,10292.0,1457.0,2316.0,875.0,7393.0,AAN
3,sub-G2004,2621.0,2329.0,6160.0,2971.0,2017.0,9591.0,13000.0,11468.0,2395.0,...,14858.0,19432.0,13154.0,11849.0,9458.0,1216.0,2477.0,854.0,7986.0,AAN
4,sub-G2005,2276.0,1949.0,7127.0,3406.0,2480.0,10771.0,9233.0,12424.0,2463.0,...,14933.0,19112.0,10911.0,10574.0,8782.0,1606.0,2971.0,842.0,6432.0,AAN


In [ ]:
aan_residuals_df = scn.create_residuals_df(aan.iloc[:,1:69], names)
aan_corr_matrix = scn.create_corrmat(aan_residuals_df, method='pearson')
aan_graph = scn.BrainNetwork(network=aan_corr_matrix, parcellation=names, centroids=centroids)
aan_graph_threshold = aan_graph.threshold(10)

In [ ]:
wr_residuals_df = scn.create_residuals_df(wr.iloc[:,1:69], names)
wr_corr_matrix = scn.create_corrmat(wr_residuals_df, method='pearson')
wr_graph = scn.BrainNetwork(network=wr_corr_matrix, parcellation=names, centroids=centroids)
wr_graph_threshold = wr_graph.threshold(10)

In [ ]:
hc_residuals_df = scn.create_residuals_df(hc.iloc[:,1:69], names)
hc_corr_matrix = scn.create_corrmat(hc_residuals_df, method='pearson')
hc_graph = scn.BrainNetwork(network=hc_corr_matrix, parcellation=names, centroids=centroids)
hc_graph_threshold = hc_graph.threshold(10)

In [ ]:
brain_bundle_aan = scn.GraphBundle([aan_graph_threshold], ['AAN_graph_thresholded'])
brain_bundle_aan.create_random_graphs('AAN_graph_thresholded', 1000)

In [ ]:
brain_bundle_wr = scn.GraphBundle([wr_graph_threshold], ['WR_graph_thresholded'])
brain_bundle_wr.create_random_graphs('WR_graph_thresholded', 1000)

In [ ]:
brain_bundle_hc = scn.GraphBundle([aan_graph_threshold], ['HC_graph_thresholded'])
brain_bundle_hc.create_random_graphs('HC_graph_thresholded', 1000)

In [ ]:
aan_global_measures = brain_bundle_aan.report_global_measures() 
aan_global_measures

In [ ]:
hc_global_measures = brain_bundle_hc.report_global_measures() 
hc_global_measures

In [ ]:
wr_global_measures = brain_bundle_wr.report_global_measures() 
wr_global_measures

In [ ]:
Pfun.distro_plots(aan_global_measures)

In [ ]:
measures = ['average_clustering', 'average_shortest_path_length', 'assortativity', 'modularity', 'efficiency'] 
measure = measures[1]
tcrit_value = aan_global_measures[measure].mean() - hc_global_measures[measure].mean() #- wr_global_measures[measure].mean()


permutations = 10000

combined = np.concatenate([aan_global_measures[measure].values, hc_global_measures[measure].values]) #wr_global_measures[measure].values, 

index_1 = len(aan_global_measures[measure].values)
#index_2 = len(wr_global_measures[measure].values) + len(wr_global_measures[measure].values)

ctvalue=[]
for perm in range(permutations):
        np.random.shuffle(combined)
        random_group_one = combined[0: index_1]
        #random_group_two = combined[index_1: index_2]
        random_group_three = combined[index_1:]
        crit_val = random_group_one.mean() - random_group_three.mean() #- random_group_two.mean() 
        ctvalue.append(crit_val)

array = np.array(ctvalue)
mean = array.mean()
std_dev = np.std(ctvalue)
lower_2std = mean - 2*std_dev
upper_2std =  mean + 2*std_dev
lower_1std = mean - std_dev
upper_1std = mean + std_dev

if tcrit_value >= mean:
        sum = [val for val in ctvalue if val >= tcrit_value]
        print('upper')
else:
        sum = [val for val in ctvalue if val <= tcrit_value]
        print('lower')
     
     #if tcrit_value > 0 and mean > 0:
     #           sum = [val for val in ctvalue if abs(val) <= abs(tcrit_value)]
     #           print('lower 1')
     #else:
     #           sum = [val for val in ctvalue if abs(val) >= abs(tcrit_value)]
     #           print('lower 2')

        
        
pval = len(sum) / permutations



print(pval)

print(tcrit_value)
print([lower_2std, upper_2std])
hist = sns.displot(data=ctvalue, height=10).set(title=f'Histplot for {measure} after {permutations} permuatations')
hist.refline(x=tcrit_value, color='purple')
hist.refline(x=lower_2std, linestyle='-', color='black')
hist.refline(x=upper_2std, linestyle='-', color='black')
hist.refline(x=lower_1std, linestyle='-', color='red')
hist.refline(x=upper_1std, linestyle='-', color='red')
plt.show()

In [ ]:
measures = ['average_clustering', 'average_shortest_path_length', 'assortativity', 'modularity', 'efficiency'] 
measure = measures[4]

tcrit_value = aan_global_measures[measure].mean() - wr_global_measures[measure].mean() - hc_global_measures[measure].mean() 

permutations = 10000

combined = np.concatenate([aan_global_measures[measure].values, wr_global_measures[measure].values, hc_global_measures[measure].values]) 

index_1 = len(aan_global_measures[measure].values)
index_2 = len(wr_global_measures[measure].values) + len(wr_global_measures[measure].values)

ctvalue=[]

for perm in range(permutations):
        np.random.shuffle(combined)
        random_group_one = combined[0: index_1]
        random_group_two = combined[index_1: index_2]
        random_group_three = combined[index_1:]
        crit_val = random_group_one.mean() - random_group_two.mean() - random_group_three.mean()  
        ctvalue.append(crit_val)

mean_val = Sfun.mean_std(ctvalue)


if tcrit_value >= mean_val['mean']:
        sum = [val for val in ctvalue if val >= tcrit_value]
        print('upper')
else:
        sum = [val for val in ctvalue if val <= tcrit_value]
        print('lower')
     
     #if tcrit_value > 0 and mean > 0:
     #           sum = [val for val in ctvalue if abs(val) <= abs(tcrit_value)]
     #           print('lower 1')
     #else:
     #           sum = [val for val in ctvalue if abs(val) >= abs(tcrit_value)]
     #           print('lower 2')

        
        
pval = len(sum) / permutations



print(pval)

print(abs(tcrit_value))
print(abs(mean_val['mean']))
Pfun.pval_plotting(ctvalue, tcrit_value, measure, permutations)


In [ ]:
def pval_plotting(ctvalues, tcrit_value, measure, permutations):
    
    '''
    Wrapper around seaborn displot.
    Function to plot null distribution with 2 std either side and true critical value.

    Parameters
    -----------------------------------------------------------------
    ctvalues : list, list of crticial values from the null distrubtion.
    tcrit_value: int True crtical value of a test.
    measure: str, name of the measure (used in title of the graph)
    permutations: int, number of permuations (used in title of the graph)


    Returns
    -----------------------------------------------------------------
    hist: histogram of null distribution with upper and lower std with
          true value.
    '''
    
    #Calculates the mean and std deviation of null distribution
    array = np.array(ctvalue)
    mean = array.mean()
    std_dev = np.std(ctvalue)

    lower_1std = mean - std_dev
    upper_1std = mean + std_dev
    lower_2std = mean - 2*std_dev
    upper_2std =  mean + 2*std_dev
    
    #plots 
    hist = sns.displot(data=ctvalues, height=10).set(title=f'Histplot for {measure} after {permutations} permuatations')
    hist.refline(x=tcrit_value, color='purple')
    hist.refline(x=lower_2std, linestyle='-', color='black')
    hist.refline(x=upper_2std, linestyle='-', color='black')
    hist.refline(x=lower_1std, linestyle='-', color='red')
    hist.refline(x=upper_1std, linestyle='-', color='red')
    plt.show()

In [ ]:
def monte_carlo(group_one, group_two, group_three, permutations, plotting=False, measure='None'):
    
    '''
    
    Function to run a monte_carlo permuation hypothesis testing.
    
    Calculates a critical value then randomly shuffles groups calculating the crticial value 
    for each permutation.

    Pval calculated as:
    sum of all values greater than critical value / number of permutations

    Parameters
    ----------------------------------------------
    group one: numpy array of values for group one
    group one: numpy array of values for group two
    group one: numpy array of values for group three
    permutations: int, number of permutations to perform.

    Returns
    ---------------------------------------------
    results: dict object with critical value, pvalue and std
    
    '''

    critical_value = group_one.mean() - group_two.mean() - group_three.mean()
    combined = np.concatenate([group_one, group_two, group_three])

    index_1 = len(group_one)
    index_2 = len(group_two) + len(group_two)

    ctvalue = []

    for perm in range(permutations):
        np.random.shuffle(combined)
        random_group_one = combined[0: index_1]
        random_group_two = combined[index_1: index_2]
        random_group_three = combined[index_2:]
        crit_val = random_group_one.mean() - random_group_two.mean() - random_group_three.mean()
        ctvalue.append(crit_val)
    
    array = np.array(ctvalue)
    mean = array.mean()
    std_dev = np.std(ctvalue)
    lower_2std = mean - 2*std_dev
    upper_2std =  mean + 2*std_dev

    if tcrit_value >= mean:
            sum = [val for val in ctvalue if val >= tcrit_value]

    else:
            sum = [val for val in ctvalue if val <= tcrit_value]
             
    pval = len(sum) / permutations



print(pval)

print(abs(tcrit_value))
print(abs(mean))
#print([lower_2std, upper_2std])
hist = sns.displot(data=ctvalue, height=10).set(title=f'Histplot for {measure} after {permutations} permuatations')
hist.refline(x=tcrit_value, color='purple')
hist.refline(x=lower_2std, linestyle='-', color='black')
hist.refline(x=upper_2std, linestyle='-', color='black')
plt.show()

    result = {'critval':critical_value, 'pval': pval,'lower_std':lower_2std, 'upper_std':upper_2std}

    if plotting == True:
        pval_plotting(ctvalue, critical_value, measure, permutations)

    return result

In [ ]:
measure = 'assortativity' 
plots_for_pval = monte_carlo(aan_global_measures[measure].values, wr_global_measures[measure].values, hc_global_measures[measure].values, permutations=50000, plotting=True, measure=measure)

In [ ]:
for measure in aan_global_measures.columns: 
    eff = monte_carlo(aan_global_measures[measure].values, wr_global_measures[measure].values, hc_global_measures[measure].values, permutations=50000)
    print(eff)

In [ ]:
def monte_carlo_2(group_one, group_two, permutations):
    
    '''
    Function to run a monte_carlo permuation hypothesis testing.
    
    Calculates a critical value then randomly shuffles groups calculating the crticial value 
    for each permutation.

    Pval calculated as:
    sum of all values greater than critical value / number of permutations

    Parameters
    ----------------------------------------------
    group one: numpy array of values for group one
    group one: numpy array of values for group two
    group one: numpy array of values for group three
    permutations: int, number of permutations to perform.

    Returns
    ---------------------------------------------
    results: dict object with critical value and pvalue
    
    '''

    critical_value = group_one.mean() - group_two.mean()
    combined = np.concatenate([group_one, group_two])

    ctvalue = []

    index_1 = len(group_one)

    for perm in range(permutations):
        np.random.shuffle(combined)
        random_group_one = combined[:index_1]
        random_group_two = combined[index_1:]
        crit_val = random_group_one.mean() - random_group_two.mean() 
        ctvalue.append(crit_val)
    
    sum = [val for val in ctvalue if abs(val) >= abs(critical_value)]
    
    #print(len(sum))
    #print('\n', ctvalue,'\n')

    pval = len(sum) / permutations

    result = {'critval':critical_value, 'pval': pval}

    return result

In [ ]:
for measure in aan_global_measures.columns: 
    eff = monte_carlo_2(aan_global_measures[measure].values, hc_global_measures[measure].values, permutations=50000)
    print(f'Group difference between AAN and HC for {measure}','pval=', eff['pval'],'critical value:',eff['critval'])

In [ ]:
for measure in aan_global_measures.columns: 
    eff = monte_carlo_2(aan_global_measures[measure].values, wr_global_measures[measure].values, permutations=50000)
    print(f'Group difference between AAN and WR for {measure}','pval=', eff['pval'],'critical value:',eff['critval'])

In [ ]:
for measure in aan_global_measures.columns: 
    eff = monte_carlo_2(hc_global_measures[measure].values, wr_global_measures[measure].values, permutations=50000)
    print(f'Group difference between HC and WR for {measure}','pval=', eff['pval'],'critical value:',eff['critval'])